In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import Doc
from functools import partial 
import numpy as np
import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import Doc
from functools import partial

import time

# Load the data
df = pd.read_csv("data/bbc-text.csv")


df["text"]=df["text"].apply(lambda x: re.sub(r'[^\w\s]', ' ',x) ) 


def log_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()  # Record start time
        result = func(*args, **kwargs)  # Execute the wrapped function
        end_time = time.time()  # Record end time
        elapsed_time = end_time - start_time
        print(f"Function '{func.__name__}' executed in {elapsed_time:.4f} seconds")
        return result
    return wrapper
 
nlp = spacy.load('en_core_web_lg')

# Disable unnecessary pipeline components
nlp.disable_pipes(["parser", "ner"])

# Custom tokenizer to speed up processing
def custom_tokenizer(text):
    return Doc(nlp.vocab, words=text.split())

nlp.tokenizer = custom_tokenizer

# Preprocess function
def preprocess_text(text, nlp, max_words=150):
    doc = nlp(text)
    tokens = [token.text for token in doc if not token.is_stop and token.text.strip()]
    return tokens[:max_words] + ['<PAD>'] * (max_words - len(tokens))
# Vectorize function
def vectorize_text(tokens, nlp):
    return np.array([nlp.vocab[token].vector for token in tokens])

# Apply preprocessing in batches
batch_size = 1000
preprocessed_texts = []

for i in tqdm(range(0, len(df), batch_size)):
    batch = df['text'].iloc[i:i+batch_size]
    preprocessed_batch = [preprocess_text(text, nlp) for text in nlp.pipe(batch, batch_size=64)]
    preprocessed_texts.extend(preprocessed_batch)

df['processed_text'] = preprocessed_texts 

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:04<00:00, 21.61s/it]


In [34]:

df

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [02:18<00:00, 46.31s/it]


,category,text,processed_text
0,tech,tv future in the hands of viewers with home th...,"[tv, future, hands, viewers, home, theatre, sy..."
1,business,worldcom boss left books alone former worldc...,"[worldcom, boss, left, books, worldcom, boss, ..."
2,sport,tigers wary of farrell gamble leicester say ...,"[tigers, wary, farrell, gamble, leicester, rus..."
3,sport,yeading face newcastle in fa cup premiership s...,"[yeading, face, newcastle, fa, cup, premiershi..."
4,entertainment,ocean s twelve raids box office ocean s twelve...,"[ocean, s, raids, box, office, ocean, s, crime..."
...,...,...,...
2220,business,cars pull down us retail figures us retail sal...,"[cars, pull, retail, figures, retail, sales, f..."
2221,politics,kilroy unveils immigration policy ex chatshow ...,"[kilroy, unveils, immigration, policy, ex, cha..."
2222,entertainment,rem announce new glasgow concert us band rem h...,"[rem, announce, new, glasgow, concert, band, r..."
2223,politics,how political squabbles snowball it s become c...,"[political, squabbles, snowball, s, commonplac..."


In [4]:
import re
import cupy as cp 
import pickle
class Word2Vec:

    def __init__(self, embedding_size, semi_context_window, complete_text,optimizer,learning_rate=0.01,flattening_strategy="concat",
                 momentum_beta=0.9,
                 rmsprop_beta=0.98,
                 epsilon=0.00000001,
                 load_pretrained=False):
        self.embedding_size = embedding_size
        self.side_window_size = semi_context_window
        self.complete_text = complete_text
        self.vocabulary = self.create_vocabulary(complete_text)
        self.flattening_strategy = flattening_strategy
        
        if load_pretrained == False:
            self.words_len_embedding_layer = cp.random.rand(len(self.vocabulary), embedding_size)
            self.words_len_embedding_bias = cp.random.rand(embedding_size)
            
    
            self.outlayer_maps_vocab_average = cp.random.rand(embedding_size, len(self.vocabulary))
            self.out_bias_maps_vocab_average = cp.random.rand(len(self.vocabulary))
    
            self.outlayer_maps_vocab_concat = cp.random.rand(semi_context_window * 2 * embedding_size, len(self.vocabulary))
            self.out_bias_maps_vocab_concat = cp.random.rand(len(self.vocabulary))
        else:
            self.load_weights()
            
        self.optimizer = optimizer
        self.learning_rate = learning_rate
        self.RMSPDW2 = 0
        self.RMSPb2 = 0 
        self.RMSPDW1 = 0
        self.RMSPb1 = 0 
        self.MomDW2 = 0
        self.Momb2  = 0
        self.MomDW1 = 0
        self.Momb1 = 0 
        self.momentum_beta=momentum_beta
        self.rmsprop_beta=rmsprop_beta
        self.epsilon=epsilon
        self.iterations=0
        self.total_loss = 0

    def cross_entropy_loss(self, predictions, target):
        # Cross-entropy loss for a batch of predictions and targets
        batch_loss = -cp.sum(target * cp.log(predictions + 1e-9), axis=1)
        return cp.mean(batch_loss)

    def softmax(self, x, axis=-1):
        x = cp.clip(x, -1e4, 1e4)  # Clip for numerical stability
        e_x = cp.exp(x - cp.max(x, axis=axis, keepdims=True))
        return e_x / cp.sum(e_x, axis=axis, keepdims=True)

    def input_one_hot_vectors(self,words,vocabulary):
     
        inputs=np.zeros((len(words),len(vocabulary)),int)
         
        for i in range(len(words)): 
            inputs[i][vocabulary[words[i]]]=1
        
        return inputs 

    def inputs_window_words(self, input_text, vocabulary):
        sequence = re.sub(r'[^\w\s]', ' ', input_text).split()
        training_samples = []
        for i in range(self.side_window_size, len(sequence) - self.side_window_size):
            words_before = sequence[i - self.side_window_size:i]
            words_after = sequence[i + 1:i + 1 + self.side_window_size]
            # print(words_before)
            # print(words_after)
            words_input = words_before + words_after
            X_i = self.input_one_hot_vectors(words_input, vocabulary)
            y_i = self.input_one_hot_vectors([sequence[i]], vocabulary)
            # print(words_input,sequence[i])
            training_sample = [X_i, y_i]
            training_samples.append(training_sample)
        yield training_samples

    def forward(self, Input):
        sigma_zout_one = self.softmax(
            cp.matmul(cp.array(Input), self.words_len_embedding_layer) + self.words_len_embedding_bias)
        
        if self.flattening_strategy == "concatenate":
            sigma_zout_one = sigma_zout_one.reshape(1, sigma_zout_one.shape[0] * self.embedding_size)
            sigma_zout_output = self.softmax(
                cp.matmul(sigma_zout_one, self.outlayer_maps_vocab_concat) + self.out_bias_maps_vocab_concat)


        elif self.flattening_strategy == "average":
            sigma_zout_one = cp.mean(sigma_zout_one, axis=0).reshape(1, sigma_zout_one.shape[1])
            sigma_zout_output = self.softmax(
                cp.matmul(sigma_zout_one, self.outlayer_maps_vocab_average) + self.out_bias_maps_vocab_average)

        return [sigma_zout_one, sigma_zout_output]

    def training_validation(self, X_validation, y_validation, current_loss):
        # Pad validation sequences
        X_validation = self.pad_sequences(X_validation, self.words_per_phrase)

        # Make predictions
        predictions = self.predict(X_validation)

        # Convert predictions to class labels
        y_pred = cupy.argmax(predictions, axis=1)

        # Convert one-hot encoded true labels to class labels
        y_true = cupy.argmax(y_validation, axis=1)

        #print("y_true",np.array(y_true.get()))
        # Calculate F1 score
        f1 = f1_score(y_true.get(), y_pred.get(), average="weighted")
        print("Validation F1 score: ", f1)

        
    def backpropagation(self, dLoss_dZ2, sigma_Z_1, sigma_Z_2, Xi):
        # print("dLoss_dZ2", dLoss_dZ2.shape)
        # print("W2", self.outlayer_maps_vocab_concat.shape)
        # print("b2", self.out_bias_maps_vocab_concat.shape)
        # print("sigma_Z_1", sigma_Z_1.shape)
        # print("sigma_Z_2", sigma_Z_2.shape)
        # print("W1", self.words_len_embedding_layer.shape)
        # print("b1", self.words_len_embedding_bias.shape)
        # print("Xi", Xi.shape)

        if self.flattening_strategy == "concatenate":



            #Gradient Embedding Weights C
            dL_dSigmaZ1=cp.matmul(dLoss_dZ2,self.outlayer_maps_vocab_concat.T)
            dL_dZ1=dL_dSigmaZ1*sigma_Z_1*(1-sigma_Z_1)
            dL_dZ1=dL_dZ1.reshape(2*self.side_window_size,self.embedding_size)
            dLoss_dW1=cp.matmul(dL_dZ1.T,Xi).T 
        

            #Gradient Embedding Bias C
            dLoss_db1=cp.sum(dL_dZ1,axis=0).T
           
            #Gradient Embedding W2C
            dLoss_dW2=cp.matmul(dLoss_dZ2.T,sigma_Z_1).T 
          

            #Gradient Embedding B2C
            dLoss_db2=dLoss_dZ2.T 
            
            # print("dLoss_dW2",dLoss_dW2.shape,"W2",self.outlayer_maps_vocab_concat.shape)
            # print("dLoss_db2",dLoss_db2.shape,"B2",self.out_bias_maps_vocab_concat.shape)
            # print("dLoss_dW1",dLoss_dW1.shape,"W1",self.words_len_embedding_layer.shape)
            # print("dLoss_db1",dLoss_db1.shape,"B1",self.words_len_embedding_bias.shape)
            # Update weights and biases
            self.update_params(dLoss_dW2,dLoss_db2,dLoss_dW1,dLoss_db1)

             
        elif self.flattening_strategy == "average":
            
            #Gradient Embedding Weights
            dL_dSigmaZ1=cp.matmul(dLoss_dZ2,self.outlayer_maps_vocab_average.T)
            dL_dSigmaZ1=dL_dSigmaZ1.repeat(2*self.side_window_size,axis=0)
            dL_dZ1=dL_dSigmaZ1*sigma_Z_1*(1-sigma_Z_1)
            dLoss_dW1=cp.matmul(dL_dZ1.T,Xi).T
           

            #Gradient Embedding Bias
            dLoss_db1=cp.sum(dL_dZ1,axis=0).T
            
            #Gradient Embedding W2
            dLoss_dW2=cp.matmul(dLoss_dZ2.T,sigma_Z_1).T
           
            
            #Gradient Embedding B2
            dLoss_db2=dLoss_dZ2.T
            
            # print("dLoss_dW2",dLoss_dW2.shape,"W2",self.outlayer_maps_vocab_average.shape)
            # print("dLoss_db2",dLoss_db2.shape,"B2",self.out_bias_maps_vocab_average.shape)
            # print("dLoss_dW1",dLoss_dW1.shape,"W1",self.words_len_embedding_layer.shape)
            # print("dLoss_db1",dLoss_db1.shape,"B1",self.words_len_embedding_bias.shape)
            # Update weights and biases
            self.update_params(dLoss_dW2,dLoss_db2,dLoss_dW1,dLoss_db1)
            
            
        else:
            print("Please select valid flattening strategy!")
         

    
    def create_vocabulary(self,complete_text):
        
        text = re.sub(r'[^\w\s]', ' ', complete_text).split()
        vocabulary = list(set(text)) 
        vocabulary = {word: idx for idx, word in enumerate(vocabulary)} 
        print("Vocabulary size: ", len(vocabulary))
        with open('data/vocabulary.pkl', 'wb') as handle:
            pickle.dump(vocabulary, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    
        return vocabulary

    def save_checkpoint(self):
     
        with open('data/words_len_embedding_layer.pkl', 'wb') as handle:
            pickle.dump(self.words_len_embedding_layer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
        with open('data/words_len_embedding_bias.pkl', 'wb') as handle:
            pickle.dump(self.words_len_embedding_bias, handle, protocol=pickle.HIGHEST_PROTOCOL)
            
            
        if self.flattening_strategy == "concatenate":
            with open('data/outlayer_maps_vocab_concat.pkl', 'wb') as handle:
                pickle.dump(self.outlayer_maps_vocab_concat, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
            with open('data/out_bias_maps_vocab_concat.pkl', 'wb') as handle:
                pickle.dump(self.out_bias_maps_vocab_concat, handle, protocol=pickle.HIGHEST_PROTOCOL)
                
    
        if self.flattening_strategy == "average":
            with open('data/outlayer_maps_vocab_average.pkl', 'wb') as handle:
                pickle.dump(self.outlayer_maps_vocab_average, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
            with open('data/out_bias_maps_vocab_average.pkl', 'wb') as handle:
                pickle.dump(self.out_bias_maps_vocab_average, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def load_weights(self):

        with open('data/words_len_embedding_layer.pkl', 'rb') as f:
            self.words_len_embedding_layer = pickle.load(f)

        with open('data/words_len_embedding_bias.pkl', 'rb') as f:
            self.words_len_embedding_bias = pickle.load(f)

        if self.flattening_strategy == "average":
            
            with open('data/outlayer_maps_vocab_average.pkl', 'rb') as f:
                self.outlayer_maps_vocab_average = pickle.load(f)
    
            with open('data/out_bias_maps_vocab_average.pkl', 'rb') as f:
                self.out_bias_maps_vocab_average = pickle.load(f)
            
            
        if self.flattening_strategy == "concatenate":
            with open('data/outlayer_maps_vocab_concat.pkl', 'rb') as f:
                self.outlayer_maps_vocab_concat = pickle.load(f)
    
            with open('data/out_bias_maps_vocab_concat.pkl', 'rb') as f:
                self.out_bias_maps_vocab_concat = pickle.load(f)

    def calculate_momentum_terms(self,dLoss_dW2,dLoss_db2,dLoss_dW1,dLoss_db1):
        # Update biased first moments (momentum)
        self.MomDW2 = (self.momentum_beta * self.MomDW2 + (1 - self.momentum_beta) * dLoss_dW2)
        self.Momb2 = (self.momentum_beta * self.Momb2 + (1 - self.momentum_beta) * dLoss_db2)
        self.MomDW1 = (self.momentum_beta * self.MomDW1 + (1 - self.momentum_beta) * dLoss_dW1)
        self.Momb1 = (self.momentum_beta * self.Momb1 + (1 - self.momentum_beta) * dLoss_db1)
        self.MomDW2_corrected = self.MomDW2 / (1 - self.momentum_beta ** self.iterations)
        self.Momb2_corrected = self.Momb2 / (1 - self.momentum_beta ** self.iterations)
        self.MomDW1_corrected = self.MomDW1 / (1 - self.momentum_beta ** self.iterations)
        self.Momb1_corrected = self.Momb1 / (1 - self.momentum_beta ** self.iterations)

    def calculate_rmsprom_terms(self,dLoss_dW2,dLoss_db2,dLoss_dW1,dLoss_db1):
        # Update biased second moments (variance)
        self.RMSPDW2 = (self.rmsprop_beta * self.RMSPDW2 + (1 - self.rmsprop_beta) * dLoss_dW2 * dLoss_dW2)
        self.RMSPb2 = (self.rmsprop_beta * self.RMSPb2 + (1 - self.rmsprop_beta) * dLoss_db2 * dLoss_db2)
        self.RMSPDW1 = (self.rmsprop_beta * self.RMSPDW1 + (1 - self.rmsprop_beta) * dLoss_dW1 * dLoss_dW1)
        self.RMSPb1 = (self.rmsprop_beta * self.RMSPb1 + (1 - self.rmsprop_beta) * dLoss_db1 * dLoss_db1)
        # Bias correction for second moments
        self.RMSPDW2_corrected = self.RMSPDW2 / (1 - self.rmsprop_beta ** self.iterations)
        self.RMSPb2_corrected = self.RMSPb2 / (1 - self.rmsprop_beta ** self.iterations)
        self.RMSPDW1_corrected = self.RMSPDW1 / (1 - self.rmsprop_beta ** self.iterations)
        self.RMSPb1_corrected = self.RMSPb1 / (1 - self.rmsprop_beta ** self.iterations)

     
    def update_params(self,dLoss_dW2,dLoss_db2,dLoss_dW1,dLoss_db1):
         
        self.iterations += 1
        
        # # Linear learning rate decay
        if self.iterations>150000:
            self.learning_rate=0.05
        if self.iterations>500000:
            self.learning_rate=0.01
        if self.iterations>750000:
            self.learning_rate=0.005
        if self.iterations>1000000:
            self.learning_rate=0.001
        if self.iterations>1250000:
            self.learning_rate=0.0001    
        # lr = self.learning_rate * (1 - self.iterations / self.total_iterations)
        # self.learning_rate = max(lr, 0.001)  # Prevent learning rate from becoming too small
        
        
        if self.optimizer=="adam":
            
            self.calculate_momentum_terms(dLoss_dW2,dLoss_db2,dLoss_dW1,dLoss_db1)
            self.calculate_rmsprom_terms(dLoss_dW2,dLoss_db2,dLoss_dW1,dLoss_db1) 

            
            # Update parameters using bias-corrected moments
            
            self.words_len_embedding_layer -= self.learning_rate * self.MomDW1_corrected / (cp.sqrt(self.RMSPDW1_corrected) + self.epsilon)
            self.words_len_embedding_bias -= self.learning_rate * self.Momb1_corrected / (cp.sqrt(self.RMSPb1_corrected) + self.epsilon)
            
            if self.flattening_strategy == "concatenate":
                self.outlayer_maps_vocab_concat -= self.learning_rate * self.MomDW2_corrected / (cp.sqrt(self.RMSPDW2_corrected) + self.epsilon)
                self.out_bias_maps_vocab_concat -= (self.learning_rate * self.Momb2_corrected / (cp.sqrt(self.RMSPb2_corrected) + self.epsilon)).reshape(dLoss_db2.shape[0])
                
            if self.flattening_strategy == "average":
                self.outlayer_maps_vocab_average -= self.learning_rate * self.MomDW2_corrected / (cp.sqrt(self.RMSPDW2_corrected) + self.epsilon)
                self.out_bias_maps_vocab_average -= (self.learning_rate * self.Momb2_corrected / (cp.sqrt(self.RMSPb2_corrected) + self.epsilon)).reshape(dLoss_db2.shape[0])
            
        if self.optimizer=="momentum": 
            
            self.calculate_momentum_terms(dLoss_dW2,dLoss_db2,dLoss_dW1,dLoss_db1)

            self.words_len_embedding_layer -= self.learning_rate * self.MomDW1_corrected  
            self.words_len_embedding_bias -= self.learning_rate * self.Momb1_corrected

            
            if self.flattening_strategy == "concatenate":
                self.outlayer_maps_vocab_concat -= self.learning_rate * self.MomDW2_corrected  
                self.out_bias_maps_vocab_concat -= self.learning_rate * self.Momb2_corrected 
                
            if self.flattening_strategy == "average":
                self.outlayer_maps_vocab_average -= self.learning_rate * self.MomDW2_corrected  
                self.out_bias_maps_vocab_average -= self.learning_rate * self.Momb2_corrected  
                
                
            
        if self.optimizer==None: 
              
            self.words_len_embedding_layer -= self.learning_rate * dLoss_dW1  
            self.words_len_embedding_bias -= self.learning_rate * dLoss_db1  
            
            if self.flattening_strategy == "concatenate":
                self.outlayer_maps_vocab_concat -= self.learning_rate * dLoss_dW2
                self.out_bias_maps_vocab_concat -= self.learning_rate * dLoss_db2.reshape(dLoss_db2.shape[0])
                
            if self.flattening_strategy == "average":
                self.outlayer_maps_vocab_average -= self.learning_rate * dLoss_dW2 
                self.out_bias_maps_vocab_average -= self.learning_rate * dLoss_db2.reshape(dLoss_db2.shape[0])
         

 
    @log_time
    def trianiteration(self,training_samples):
        for sample in training_samples:
            
            Xi = cp.array(sample[0]) 
            
            yi = cp.array(sample[1])
            
            sigmaz_out_one, sigma_zout_two = self.forward(Xi)
            
            Loss = self.cross_entropy_loss(sigma_zout_two, yi)
            
            self.total_loss += Loss
            
            dLoss_dZout = sigma_zout_two - yi
    
            self.backpropagation(dLoss_dZout, sigmaz_out_one, sigma_zout_two, Xi)
        

    def predict(self, X):
        sigma_zout_one = self.softmax(
            cp.matmul(cp.array(X), self.words_len_embedding_layer) + self.words_len_embedding_bias)
        return sigma_zout_one

    def train(self, X_train):
        
        # if self.validation_split > 0:
        #     X_train, X_train_validation, y_train, y_train_validation = train_test_split(
        #         X_train, y_train, test_size=self.validation_split, random_state=42)
        #     print("Training starting:")
        #     print("X_train samples: ", X_train.shape[0])
        #     print("X_train_validation samples", X_train_validation.shape[0], "\n")

        
        for trainingepoch in range(50):
            self.iterations=0
            print("Epoch: ",trainingepoch)
            for i in range(len(X_train)):
                gen = self.inputs_window_words(X_train[i], self.vocabulary)
                for training_samples in gen:

                    print(f"training samples from row {i}: {len(training_samples)},iterations {self.iterations},lr {self.learning_rate}")

                    self.trianiteration(training_samples)
                        
                        
                    print(f"Loss sample: {self.total_loss / len(training_samples)}")
                    self.total_loss = 0
            self.save_checkpoint()


 



In [5]:
complete_text

NameError: name 'complete_text' is not defined

In [6]:
complete_text = ' '.join(df['processed_text'].apply(lambda x: ' '.join(x)).tolist()) 
#complete_text = ' '.join(df["text"].tolist()) 
embedding_size=100
semi_context_window=5 

model=Word2Vec(embedding_size, 
               semi_context_window, 
               complete_text,
               optimizer=None,
               learning_rate=0.0005,
               flattening_strategy="average",
               load_pretrained=False)

model.train(df['processed_text'].apply(lambda x: ' '.join(x)).tolist())

Vocabulary size:  24040
Epoch:  0
training samples from row 0: 140,iterations 0,lr 0.0005
Function 'trianiteration' executed in 0.5048 seconds
Loss sample: 10.132353232091553
training samples from row 1: 140,iterations 140,lr 0.0005
Function 'trianiteration' executed in 0.3785 seconds
Loss sample: 10.105676746761674
training samples from row 2: 140,iterations 280,lr 0.0005
Function 'trianiteration' executed in 0.3862 seconds
Loss sample: 10.149971715417308
training samples from row 3: 140,iterations 420,lr 0.0005
Function 'trianiteration' executed in 0.3725 seconds
Loss sample: 10.112176459786149
training samples from row 4: 140,iterations 560,lr 0.0005
Function 'trianiteration' executed in 0.3652 seconds
Loss sample: 10.139584025734402
training samples from row 5: 140,iterations 700,lr 0.0005
Function 'trianiteration' executed in 0.3717 seconds
Loss sample: 10.104262737717214
training samples from row 6: 140,iterations 840,lr 0.0005
Function 'trianiteration' executed in 0.3892 seconds

KeyboardInterrupt: 

In [58]:
from collections import Counter
 
Counter(' '.join(df['processed_text'].apply(lambda x: ' '.join(x)).tolist()).split())

Counter({'<PAD>': 21157,
         's': 6199,
         'said': 4763,
         'mr': 1750,
         'year': 1697,
         'new': 1350,
         'people': 1138,
         'time': 862,
         'world': 821,
         'uk': 801,
         't': 762,
         'government': 709,
         'film': 685,
         '1': 677,
         'years': 648,
         'told': 648,
         'best': 647,
         '000': 572,
         'game': 569,
         'number': 561,
         '2': 550,
         'bbc': 543,
         'labour': 531,
         'set': 511,
         '6': 499,
         'added': 485,
         'england': 479,
         'company': 478,
         'firm': 478,
         '2004': 478,
         'music': 472,
         'like': 469,
         'win': 460,
         'market': 460,
         'second': 450,
         '3': 445,
         'week': 443,
         'election': 440,
         'won': 439,
         'home': 432,
         'says': 424,
         'way': 419,
         '4': 408,
         'blair': 407,
         'million': 404,

In [ ]:
model.save_checkpoint()